<a href="https://colab.research.google.com/github/xh313/Corpus-Linguistics-Works/blob/main/CorpusLingFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Planning

## Inspiration:
![meme](https://user-images.githubusercontent.com/77285010/161146641-d099beed-56d7-40ef-8ae4-0083fe473d6f.png)

(source: https://www.reddit.com/r/Suomi/comments/tqx5ac/mielipide_suomesta_eri_alaredditeiss%C3%A4/)

When I was learning Finnish through browsing the Finnish language community on reddit r/suomi (meaning Finland in Finnish), I saw this meme which makes a contrast between the English language subreddit on Finland and the native language one. Obviously, the contrast between the sentiment on the place of the natives and the 'tourists' is pretty hilarious. I am thus intrigued to see what the differences are through analysing the corpora!


## Idea

- Scrap the English and native language subreddits of different places in the world and compare sentiments.
- Assume that the English corpora represent the discussions within 'international community', while the native corpora represent the 'local community'.
- Compare and contrast:
  - English vs Native sentiments
  - How the contrast is different for different countries / places around the world.

## Methods
- Scrap the subreddits
- Translation: first compare if the sentiment of the translated corpora is similar enough to the original corpora by sampling a few languages
- If translation does not strip the sentiment too much, the translated corpora would be used to do comparative analysis.



---


[NOT UPDATING] Channel lists:

No longer updating since this is too laborious.

E = English, N = Native

- Poland:
  - E: https://www.reddit.com/r/poland/
  - N: https://www.reddit.com/r/Polska/

- Finland:
  - E: https://www.reddit.com/r/Finland/
  - N: https://www.reddit.com/r/Suomi/

- Brazil:
  - E: https://www.reddit.com/r/Brazil/
  - N: https://www.reddit.com/r/brasil/

- Turkey:
  - E: https://www.reddit.com/r/Turkey/
  - N: https://www.reddit.com/r/Turkiye/

## Channel Lists (New):

In [5]:
# Tester list to run faster
test_srs = ['Finland', 'Suomi', 
       'Brazil', 'brasil'
       ]

# Tester list for translation analysis
trans_srs = [#'poland', 'Polska',   # The Poland sr does not work properly
       'Finland', 'Suomi', 
       'Brazil', 'brasil',
       #'Turkey', 'Turkiye',
       'China', 'China_irl',  # Uncertain if this counts
       #'japan', 'ja',
       #'Thailand', 'thaithai',
       #'spain', 'es',  # The 'spain' one is flooded w spanish as well...
       #'southafrica', 'RSA',
       ]

# Current working list
# English, Native format!!
srs = ['poland', 'Polska',   # The Poland sr does not work properly
       'Finland', 'Suomi', 
       'Brazil', 'brasil',
       'Turkey', 'Turkiye',
       'China', 'China_irl',  # Uncertain if this counts
       'japan', 'ja',
       'Thailand', 'thaithai',
       'spain', 'es',  # The 'spain' one is flooded w spanish as well...
       'southafrica', 'RSA',
       'Norway', 'norge',
       ]

Always run the list cell after adding new subreddits to make sure that the list is up-to-date

# Issues / Limitations

## To-be-resolved list

1. Some native language forums have bits and pieces of English mixed in, or just reddit codes and stuff in English.
2. How many hot posts should I extract per forum?
3. Naïve Bayes only telling the positive / negative or tone analysis instead?
4. Not-so-accurate results for highly-infleceted langs?

## Unresolved limitations

1. Size of subreddits vary a lot
2. Naturally the topic focus would be subtly different in different subreddits, which ones are comparable?

# Codes

## Installations / Mounting / Importing

### Installations

In [1]:
!pip install praw

     |████████████████████████████████| 176 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 807 kB/s 


### Importing

In [2]:
import praw

reddit = praw.Reddit(client_id="R7keGb8iIXeFuLo-CZOs7g",
                     client_secret="ieKEfzD3VvRdmg-HGR0yUt3A6jD_Lw",
                     user_agent="scrapata",
                     username='hxhthewebscraper',
                     password='Puhun_suomea4')

print(reddit.user.me())

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



hxhthewebscraper


In [3]:
import re
import numpy as np
import pandas as pd

### Mounting

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Scraping

In [6]:
def subr_to_list(subreddit, number):
  """
  Takes a subreddit name and scraps posts into a list.
  Returns the list.
  """
  subr_list = list() #key is body, value is judgement

  # get 10 hot posts from the r/Suomi subreddit
  hot_posts = reddit.subreddit(subreddit).hot(limit=number) #this is pulling out the hot posts
  for post in hot_posts:  
      submission = reddit.submission(id=post.id)
      subr_list.append(submission.selftext)
      #linked_post = reddit.submission(url='https://www.reddit/com/'+post.url)
      #for top_level_comment in submission.comments:
        #print(top_level_comment.body)
        #subr_dict[submission.selftext] = top_level_comment.body

  return subr_list

In [7]:
def list_to_txt(postlist, filename, ordinal='1'):
  """
  Takes a list of processed subreddit posts and output a .txt file.
  The ordinal in the end is added to sort the files properly.
  """
  outfile = open('/content/drive/My Drive/LGCS124/FinalProject/TestCorpora/' + str(int(ordinal)) + '_' + filename + '.txt', 'w')

  for post in postlist:
    outfile.write(post)
    outfile.write('\n')

  outfile.close()

In [8]:
# Tester: [DON'T RERUN THIS IT TAKES AGES!!!]

#finland_e = subr_to_list('Suomi', 100)

In [ ]:
#list_to_txt(finland_e, 'finland_e')

### The ultimate scraper!

In [10]:
def subr_extract(subreddit, number, ordinal='1'):
  """
  The ultimate extraction. Takes a string that is the name of the subreddit
  and the number of posts to be scraped.
  The ordinal in the end is added to sort the files properly.
  """
  postlist = subr_to_list(subreddit, number)
  list_to_txt(postlist, subreddit, ordinal)

In [ ]:
# Tester
subr_extract('Polska', 10)

In [ ]:
# Tester
ord = 1
for sr in test_srs:
  subr_extract(sr, 10, ord)
  ord += 0.5

### Function written to scrape the list [srs](https://colab.research.google.com/drive/1Fh5wP9f8Qt47cjE8xgT86ub9eWUZo-iR#scrollTo=ImvYf0MSCW4B&line=3&uniqifier=1)

In [12]:
def scrap_srs(currentsrs, number):
  ord = 1
  for sr in currentsrs:
    subr_extract(sr, number, ord)
    ord += 0.5

### Call this to get all files from current srs into the drive


In [ ]:
# DON'T RUN UNLESS YOU WANT TO GET THE FILE NOW!
#scrap_srs(srs, 30)

## Translation testing
This is to check if the sentiment would be lost after translation of the non-English corpora into English. Two translators would be used:
- DeepL
- Google Translate

To test translation, three pairs of corpora are used:
1. Finland (Finnish, English)
2. Brazil (Brazilian Portuguese, English)
3. China (Simplified Chinese, English)

Of which Finnish has the highest morphological complexity being an agglutinative language, Brazilian Portuguese the medium and Chinese the lowest. 

In [ ]:
# Run this code cell to get translation tester corpora
scrap_srs(trans_srs, 30)

For the Finnish sentiment analysis, the testing corpus used is from the Korp API: https://korp.csc.fi/. I referenced https://github.com/tjkemp/ubik-sentiment for the code used to strip data from the corpus. It is combined with another labeled corpus from Turku University.

### Corpus input

#### Turku corpus

In [31]:
df = pd.read_csv('/content/drive/MyDrive/LGCS124/FinalProject/TranslationTrainingData/FinnSentiment2020.tsv', sep='\t', header=0)

df.head()

,a,b,c,major,derive,smiley,produ,split,batch,index in original,text
0,1,0,1,1,4,0,-1,1,1,comments2008c.vrt 2145269,- Tervetuloa skotlantiin...
1,0,1,0,0,4,0,-1,12,1,comments2011c.vrt 3247745,"...... No, oikein sopiva sattumaha se vaan oli..."
2,0,0,0,0,3,0,-1,14,1,comments2007c.vrt 3792960,40.
3,1,1,1,1,5,0,1,7,1,comments2010d.vrt 2351708,Kyseessä voi olla loppuelämäsi nainen.
4,1,1,1,1,5,0,1,12,1,comments2007d.vrt 1701675,Sinne vaan ocean clubiin iskemään!


In [32]:
pos_df = df[df['derive'] > 3]
pos_list = list(pos_df['text'])

neg_df = df[df['derive'] < 3]
neg_list = list(neg_df['text'])

In [33]:
pos_list[:10]

['- Tervetuloa skotlantiin...',
 '...... No, oikein sopiva sattumaha se vaan oli, vai mitä?',
 'Kyseessä voi olla loppuelämäsi nainen.',
 'Sinne vaan ocean clubiin iskemään!',
 'Itsekin pidän Keskustan kampanjointia ihan hyvänä.',
 'Muutenkin suosittelen kaikille asiasta kiinnostuneille tuota Mark "Gravy" Robertsin mainiota paperia.',
 'Ja kun vielä nuukan puoleinenkin on, mikäs sen lepposampaa elää hänen vierellään',
 'Tottakai myös ylpeä "tulevasta" vaimostani.',
 'Suuret kiitokset tämän mahtavan tapahtuman järjestäjille!',
 'Kuka "korkkaisi" minut?']

In [34]:
neg_list[:10]

['huomista päivää odotellen..',
 'en haluaisi että kissani vuotaa.. =)',
 'Nyt olisi lääkitys paikallaan.',
 'Eniten pelkään sitä, että jos mies vain koko ajan siirtää perheenperustamista vuosilla eteenpäin, kunnes emme enää saakkaan lapsia..tiedä häntä.',
 'Teillähän asenne on kohdallaan!',
 'Tuntuu kuin olisin pelkkä huora.',
 'en kertonut, koska oisit vaan suuttunut.',
 'Missä kohtaa olen sinua nimitellyt?',
 'Onko joku tehnyt tälläisen tempun?',
 'Sateenkaariväki kuvittelee, että kaksi "isää" ei voi korvata äitiä, eikä kaksi "äitiä" voi korvata isää.']

In [43]:
# Export to file for local lemmatization

def list_to_corp_file(listname, filename):
  """
  Takes a list called listname and output a txt file with each item on one line.
  The txt file is named filename (string).
  """
  outfile = open('/content/drive/MyDrive/LGCS124/FinalProject/TranslationTrainingData/' + filename + '.txt', 'w')
  for l in listname:
    outfile.write(l)
    outfile.write('\n')
  outfile.close()

In [44]:
list_to_corp_file(pos_list, 'pos_list')
list_to_corp_file(neg_list, 'neg_list')

#### Korpi corpus (Not in use)

A corpus analysing sentiment based on the smilely tags. Stopped using since its accuracy of tagging is questionable.

In [52]:
pos_korpi = []
neg_korpi = []

In [53]:
infile = open('/content/drive/MyDrive/LGCS124/FinalProject/korp_all_sentences.txt', 'r')
all = infile.read()

infile.close()

In [54]:
cleaned = re.sub(r'\nb', r'\n', all)  # remove all the b leading each line
cleaned = re.sub('\n\'\:\)\'', '', cleaned)  # remove the lines with only :)
cleaned = re.sub('\n\'\:\(\'', '', cleaned)  # remove the lines with only :(
cleaned = re.sub('\'\n\'', '\n', cleaned)  # remove the quotation marks 
cleaned = cleaned.replace('\\xc3\\xa4', 'ä')
cleaned = cleaned.replace('\\xc3\\xb6', 'ö')
splitted = cleaned.split('\n')

for line in splitted:
  if ':)' in line:
    pos_korpi.append(line[:-3])  # Exclude the :)
  else:
    neg_korpi.append(line[:-3])

In [55]:
pos_korpi[:20]

['kitystä ei sukupuolella tai iällä ) - Olisin Charlien ja Fredin ja Georgen sekoitus',
 'Ja jos jäikin niin mielikuvitus on hoitanut homman',
 '- Hagrid',
 'Ensin ajattelin , että se on varmaan ihan huono , mutta olikin sitten hyvä',
 'tai Lily Luna Potter',
 ') Rowling kirjoittaa hyvin ja tarina on uskomaton',
 'ja ron joutuu viettää loppu ikänsä terapeutin vastaan otolla ..',
 'jotenki noin se meni',
 'njaah Jos Harry kuolee nii se pääse äitinsä , isänsä , Siriuksen ja Dumbledoren luo',
 'Jotain olin lainannut kansainvälisten foorumeiden spekulaatioista , mutta ainakin tuo Nevillen vanhempia koskeva osuus oli ihan omani',
 'Rowling onnistuu yllättämään aina !',
 'Harry ja Voldemortt Nii voihan se tietenkin olla niinkin',
 'Enkä usko , että Ginnyäkään hylätään aivan vielä ...',
 'Mutta uskon että Nevillellä tulee olemaan suuri rooli viimeisessä kirjassa , kuin myös Lunalla joka loisti poissaolollaan kirjassa 6. Loppuratkaisua en halua edes pohtia , jätän sen Jolle',
 '10. Dobby on ko

### Stemming / Lemmatizing

The stemmer is not in use since it does not increase model accuracy.

The lemmatizer is from `libvoikko`. Unfortunately, it is not supporting the colab notebook, so I would have to run a script on my local machine.

In [35]:
#Import nltk Snowball stemmer
from nltk.stem.snowball import SnowballStemmer

#Create a Finnish instance
fistemmer = SnowballStemmer("finnish")

#Print the stemmed version of some Finnish word
print(fistemmer.stem('toimittaa'))

toimit


### Naïve Bayes Training Code

In [47]:
# Import back the list of lemmas

pos_lemma_df = pd.read_csv('/content/drive/MyDrive/LGCS124/FinalProject/TranslationTrainingData/pos_lemma.csv')
neg_lemma_df = pd.read_csv('/content/drive/MyDrive/LGCS124/FinalProject/TranslationTrainingData/neg_lemma.csv')

pos_lemma = list(pos_lemma_df['lemma'])
neg_lemma = list(neg_lemma_df['lemma'])

In [50]:
# Naïve bayes training functions

def get_word_counts(word_list):
    """
    Takes as input a lemma list and returns a dictionary with the
    number of times each word occurred in that file.
    :param list: The file to be counted
    :return: A dictionary containing the number of occurrences for each word
    """

    dictionary = {}
    #word_list = []

    #for l in listname:
        #word_list += l.split()

    for word in word_list:
      word_stemmed = word  # No stemming
      #word_stemmed = fistemmer.stem(word)  # stemming
      if word_stemmed not in dictionary:
          dictionary[word_stemmed] = 1
      else:
          dictionary[word_stemmed] += 1

    return dictionary

def counts_to_probs(dictionary, num):
    """
    Takes a dictionary and a number and generates a new dictionary with
    the same keys where each value has been divided by the input number.
    :param dictionary: The dictionary to be operated on
    :param num: The number to be divided
    :return: A dictionary in which all values for each key have been divided by num
    """
    for word in dictionary:
        dictionary[word] /= num

    return dictionary

def train_model(listname, word_list):
    """
    Takes as input a listname containing examples and returns a dictionary with the word probabilities.
    :param listname: The name of the list to be counted and calculated
    :return: A dictionary containing the probability of the occurrences of each word type
    """
    lines = len(listname)
    counts = get_word_counts(word_list)
    probs = counts_to_probs(counts, lines)
    return probs

def get_probability(dictionary, string):
    """
    Takes as input two parameters, a dictionary of word probabilities and a
    string (representing a review), and returns the probability of that review
    by multiplying the probabilities of each of the words in the review.
    :param dictionary: The dictionary containing the probability data
    :param string: The string to be evaluated on
    :return: The product of teh probability of each of the words in the string
    """
    string = string.lower()
    word_list = string.split()  # Create a list of words from the input str
    probs = 1  # Initialise

    for word in word_list:
        if word in dictionary:
            probs *= dictionary[word]
        else:
            probs *= 1 / 11000

    return probs


def classify(string, pos_dict, neg_dict):
    """
    Takes a string (a review), a positive and a negative model. Returns “positive” or
    “negative” depending on which model has the highest probability for the review.
    Ties goes to positive.
    :param string: The review to be classified
    :param pos_dict: The positive model (dict)
    :param neg_dict: The negative model (dict)
    :return: “positive” or “negative” depending on which model has the highest probability for the review.
    """
    pos_probs = get_probability(pos_dict, string)
    neg_probs = get_probability(neg_dict, string)
    if (pos_probs - neg_probs) >= 0:
        return 'positive'
    else:
        return 'negative'

def sentiment_analyzer_interactive(pos_list, neg_list):
    """
    An interactive function that takes two files as input, a positive
    examples file and a negative examples file. It would train a positive
    and negative model using these files and then repeatedly ask the user
    to enter a sentence and then output the classification of that sentence
    (as positive or negative). A blank line/sentence should terminate the function.
    :param pos_file: File name of the positive training data
    :param neg_file: File name of the negative training data
    """
    pos_dict = train_model(pos_list, pos_lemma)
    neg_dict = train_model(neg_list, neg_lemma)  # Train the models

    print('Blank line terminates.')
    string = input('Enter a sentence: ')
    while string != '':
        print(classify(string, pos_dict, neg_dict))
        string = input('Enter a sentence: ')



def get_accuracy(pos_train, neg_train, pos_test, neg_test):
    """
    A function that would train the model (i.e., both positive and negative counts) and then classify
    all of the test examples (both positive and negative) and keep track of the accuracy of the
    model. It would print out three scores: the accuracy on the positive test examples,
    the accuracy on the negative test examples, and the accuracy on all of the test examples.
    :param pos_test_name: File name of the positive testing data
    :param neg_test_name: File name of the negative testing data
    :param pos_train: File name of the positive training data
    :param neg_train: File name of the negative training data
    """
    pos_dict = train_model(pos_train, pos_lemma)
    neg_dict = train_model(neg_train, neg_lemma)  # Train the models

    pos_total = len(pos_test)
    neg_total = len(neg_test)  # Total num of pos & neg reviews

    pos_actual = 0
    neg_actual = 0  # Initialise

    for l in pos_test:
        if classify(l, pos_dict, neg_dict) == 'positive':
            pos_actual += 1

    for l in neg_test:
        if classify(l, pos_dict, neg_dict) == 'negative':
            neg_actual += 1

    pos_accuracy = pos_actual / pos_total
    neg_accuracy = neg_actual / neg_total
    total_accuracy = (pos_actual + neg_actual) / (pos_total + neg_total)

    print('Positive accuracy: ', pos_accuracy)
    print('Negative accuracy: ', neg_accuracy)
    print('Total accuracy: ', total_accuracy)

In [51]:
get_accuracy(pos_list, neg_list, pos_list, neg_list)

Positive accuracy:  0.6641638225255972
Negative accuracy:  0.8457292271934922
Total accuracy:  0.7749800514229985


In [97]:
sentiment_analyzer_interactive(pos_list, neg_list)

Blank line terminates.
Enter a sentence: Jos taas mielestäsi maailmassa on jokin pielessä ja haluat muutosta, osoita tukesi niin liity mukaan muuttamaan maailmaa.
positive
Enter a sentence: 
